In [ ]:
%env CUDA_VISIBLE_DEVICES=''

In [ ]:
from boardlaw.arena.refine import *

In [ ]:
tr, args = evaluate('2021-02-06 12-16-42 wan-ticks', None, 128)

In [ ]:
xs = np.array([analysis.difference(activelo.solve(args.games, args.wins), 'mohex-0.00', 'agent') for _ in range(50)])

In [ ]:
np.save{'n': args.games.values, 'w': args.wins.values, 'rng': torch.get_rng_state().numpy()}